In [4]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf

rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# set Dropout
do = 0.3

if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
# # remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# feat = feat[mask]
# API model, if use RNN, first two rnn layer must return_sequences=True


In [6]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    #net_speech = LSTM(64, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(dropout_rate)(net_speech)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_speech) for name in target_names]

    model = Model(input_speech, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(0.1, 0.5, 0.4)
model = model()
model.summary()
# main function for for LOSO
# main function for for LOSO


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
speech_input (InputLayer)       (None, 1, 68)        0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 1, 68)        272         speech_input[0][0]               
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1, 32)        12928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 1, 64)        24832       lstm_1[0][0]                     
___________

In [3]:
# np.array(predict).shape

In [4]:
#speech_to_npy
model1 = model()
earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
hist1 = model1.fit(feat[:8000], vad[:8000].T.tolist(), epochs=50, 
                   batch_size=32, verbose=1, validation_split=0.2, 
                   callbacks=[earlystop])

eval_metrik1 = model1.evaluate(feat[8000:], vad[8000:].T.tolist())
print(eval_metrik1)

# make prediction
predict = model1.predict(feat[6400:], batch_size=32)
np.save('speech_npy', 
         np.array(predict).reshape(3, 3639).T)


In [5]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(feat[6400:], batch_size=32)
    np.save('npy8000/speech_8000-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/speech_8000-1.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 5s 744us/step - loss: 1.8807 - v_loss: 0.9271 - a_loss: 0.4074 - d_loss: 0.5462 - v_ccc: 0.0729 - a_ccc: 0.5926 - d_ccc: 0.4538 - val_loss: 2.4733 - val_v_loss: 0.8558 - val_a_loss: 0.8240 - val_d_loss: 0.7936 - val_v_ccc: 0.1442 - val_a_ccc: 0.1760 - val_d_ccc: 0.2064
Epoch 2/50
6400/6400 [==============================] - 2s 379us/step - loss: 1.6029 - v_loss: 0.7704 - a_loss: 0.3610 - d_loss: 0.4714 - v_ccc: 0.2296 - a_ccc: 0.6390 - d_ccc: 0.5286 - val_loss: 2.0975 - val_v_loss: 0.8924 - val_a_loss: 0.5888 - val_d_loss: 0.6163 - val_v_ccc: 0.1076 - val_a_ccc: 0.4112 - val_d_ccc: 0.3837
Epoch 3/50
6400/6400 [==============================] - 3s 447us/step - loss: 1.5642 - v_loss: 0.7282 - a_loss: 0.3689 - d_loss: 0.4671 - v_ccc: 0.2718 - a_ccc: 0.6311 - d_ccc: 0.5329 - val_loss: 2.0404 - val_v_loss: 0

Epoch 3/50
6400/6400 [==============================] - 3s 453us/step - loss: 1.5450 - v_loss: 0.7109 - a_loss: 0.3651 - d_loss: 0.4690 - v_ccc: 0.2891 - a_ccc: 0.6349 - d_ccc: 0.5310 - val_loss: 2.0529 - val_v_loss: 0.9002 - val_a_loss: 0.5432 - val_d_loss: 0.6095 - val_v_ccc: 0.0998 - val_a_ccc: 0.4568 - val_d_ccc: 0.3905
Epoch 4/50
6400/6400 [==============================] - 3s 462us/step - loss: 1.5120 - v_loss: 0.6880 - a_loss: 0.3592 - d_loss: 0.4648 - v_ccc: 0.3120 - a_ccc: 0.6408 - d_ccc: 0.5352 - val_loss: 2.0421 - val_v_loss: 0.8822 - val_a_loss: 0.5435 - val_d_loss: 0.6163 - val_v_ccc: 0.1178 - val_a_ccc: 0.4565 - val_d_ccc: 0.3837
Epoch 5/50
6400/6400 [==============================] - 3s 433us/step - loss: 1.4001 - v_loss: 0.5983 - a_loss: 0.3434 - d_loss: 0.4584 - v_ccc: 0.4017 - a_ccc: 0.6566 - d_ccc: 0.5416 - val_loss: 1.8778 - val_v_loss: 0.8452 - val_a_loss: 0.4584 - val_d_loss: 0.5743 - val_v_ccc: 0.1548 - val_a_ccc: 0.5416 - val_d_ccc: 0.4257
Epoch 6/50
6400/6400 [

Epoch 10/50
6400/6400 [==============================] - 3s 394us/step - loss: 1.2524 - v_loss: 0.4995 - a_loss: 0.3188 - d_loss: 0.4341 - v_ccc: 0.5005 - a_ccc: 0.6812 - d_ccc: 0.5659 - val_loss: 1.8564 - val_v_loss: 0.8168 - val_a_loss: 0.4609 - val_d_loss: 0.5787 - val_v_ccc: 0.1832 - val_a_ccc: 0.5391 - val_d_ccc: 0.4213
Epoch 11/50
6400/6400 [==============================] - 3s 393us/step - loss: 1.2253 - v_loss: 0.4905 - a_loss: 0.3136 - d_loss: 0.4211 - v_ccc: 0.5095 - a_ccc: 0.6864 - d_ccc: 0.5789 - val_loss: 1.8374 - val_v_loss: 0.8020 - val_a_loss: 0.4552 - val_d_loss: 0.5802 - val_v_ccc: 0.1980 - val_a_ccc: 0.5448 - val_d_ccc: 0.4198
Epoch 12/50
6400/6400 [==============================] - 3s 441us/step - loss: 1.2212 - v_loss: 0.4753 - a_loss: 0.3182 - d_loss: 0.4277 - v_ccc: 0.5247 - a_ccc: 0.6818 - d_ccc: 0.5723 - val_loss: 1.8162 - val_v_loss: 0.7969 - val_a_loss: 0.4478 - val_d_loss: 0.5716 - val_v_ccc: 0.2031 - val_a_ccc: 0.5522 - val_d_ccc: 0.4284
Epoch 13/50
6400/64

Epoch 13/50
6400/6400 [==============================] - 2s 341us/step - loss: 1.1901 - v_loss: 0.4592 - a_loss: 0.3111 - d_loss: 0.4198 - v_ccc: 0.5408 - a_ccc: 0.6889 - d_ccc: 0.5802 - val_loss: 1.8806 - val_v_loss: 0.8345 - val_a_loss: 0.4668 - val_d_loss: 0.5793 - val_v_ccc: 0.1655 - val_a_ccc: 0.5332 - val_d_ccc: 0.4207
Epoch 14/50
6400/6400 [==============================] - 2s 315us/step - loss: 1.1662 - v_loss: 0.4516 - a_loss: 0.3024 - d_loss: 0.4123 - v_ccc: 0.5484 - a_ccc: 0.6976 - d_ccc: 0.5877 - val_loss: 1.8661 - val_v_loss: 0.8436 - val_a_loss: 0.4506 - val_d_loss: 0.5719 - val_v_ccc: 0.1564 - val_a_ccc: 0.5494 - val_d_ccc: 0.4281
Epoch 15/50
6400/6400 [==============================] - 2s 317us/step - loss: 1.1580 - v_loss: 0.4362 - a_loss: 0.3054 - d_loss: 0.4164 - v_ccc: 0.5638 - a_ccc: 0.6946 - d_ccc: 0.5836 - val_loss: 1.8549 - val_v_loss: 0.8159 - val_a_loss: 0.4600 - val_d_loss: 0.5789 - val_v_ccc: 0.1841 - val_a_ccc: 0.5400 - val_d_ccc: 0.4211
Epoch 16/50
6400/64

Epoch 8/50
6400/6400 [==============================] - 3s 471us/step - loss: 1.2604 - v_loss: 0.5034 - a_loss: 0.3223 - d_loss: 0.4347 - v_ccc: 0.4966 - a_ccc: 0.6777 - d_ccc: 0.5653 - val_loss: 1.8958 - val_v_loss: 0.8406 - val_a_loss: 0.4638 - val_d_loss: 0.5915 - val_v_ccc: 0.1594 - val_a_ccc: 0.5362 - val_d_ccc: 0.4085
Epoch 9/50
6400/6400 [==============================] - 3s 467us/step - loss: 1.2539 - v_loss: 0.5056 - a_loss: 0.3153 - d_loss: 0.4330 - v_ccc: 0.4944 - a_ccc: 0.6847 - d_ccc: 0.5670 - val_loss: 1.8760 - val_v_loss: 0.8272 - val_a_loss: 0.4693 - val_d_loss: 0.5795 - val_v_ccc: 0.1728 - val_a_ccc: 0.5307 - val_d_ccc: 0.4205
Epoch 10/50
6400/6400 [==============================] - 3s 460us/step - loss: 1.2389 - v_loss: 0.4889 - a_loss: 0.3196 - d_loss: 0.4304 - v_ccc: 0.5111 - a_ccc: 0.6804 - d_ccc: 0.5696 - val_loss: 1.8860 - val_v_loss: 0.8548 - val_a_loss: 0.4505 - val_d_loss: 0.5808 - val_v_ccc: 0.1452 - val_a_ccc: 0.5495 - val_d_ccc: 0.4192
Epoch 11/50
6400/6400

...

Epoch 5/50
6400/6400 [==============================] - 3s 508us/step - loss: 1.4834 - v_loss: 0.6630 - a_loss: 0.3577 - d_loss: 0.4627 - v_ccc: 0.3370 - a_ccc: 0.6423 - d_ccc: 0.5373 - val_loss: 1.9781 - val_v_loss: 0.8602 - val_a_loss: 0.4966 - val_d_loss: 0.6213 - val_v_ccc: 0.1398 - val_a_ccc: 0.5034 - val_d_ccc: 0.3787
Epoch 6/50
6400/6400 [==============================] - 3s 517us/step - loss: 1.3524 - v_loss: 0.5606 - a_loss: 0.3373 - d_loss: 0.4544 - v_ccc: 0.4394 - a_ccc: 0.6627 - d_ccc: 0.5456 - val_loss: 1.8620 - val_v_loss: 0.8383 - val_a_loss: 0.4597 - val_d_loss: 0.5639 - val_v_ccc: 0.1617 - val_a_ccc: 0.5403 - val_d_ccc: 0.4361
Epoch 7/50
6400/6400 [==============================] - 3s 543us/step - loss: 1.3134 - v_loss: 0.5314 - a_loss: 0.3278 - d_loss: 0.4542 - v_ccc: 0.4686 - a_ccc: 0.6722 - d_ccc: 0.5458 - val_loss: 1.8877 - val_v_loss: 0.8542 - val_a_loss: 0.4632 - val_d_loss: 0.5703 - val_v_ccc: 0.1458 - val_a_ccc: 0.5368 - val_d_ccc: 0.4297
Epoch 8/50
6400/6400 [

Epoch 5/50
6400/6400 [==============================] - 3s 531us/step - loss: 1.4980 - v_loss: 0.6773 - a_loss: 0.3544 - d_loss: 0.4663 - v_ccc: 0.3227 - a_ccc: 0.6456 - d_ccc: 0.5337 - val_loss: 1.9794 - val_v_loss: 0.8769 - val_a_loss: 0.4870 - val_d_loss: 0.6155 - val_v_ccc: 0.1231 - val_a_ccc: 0.5130 - val_d_ccc: 0.3845
Epoch 6/50
6400/6400 [==============================] - 3s 526us/step - loss: 1.4040 - v_loss: 0.6113 - a_loss: 0.3343 - d_loss: 0.4585 - v_ccc: 0.3887 - a_ccc: 0.6657 - d_ccc: 0.5415 - val_loss: 1.9040 - val_v_loss: 0.8430 - val_a_loss: 0.4679 - val_d_loss: 0.5930 - val_v_ccc: 0.1570 - val_a_ccc: 0.5321 - val_d_ccc: 0.4070
Epoch 7/50
6400/6400 [==============================] - 3s 534us/step - loss: 1.3397 - v_loss: 0.5541 - a_loss: 0.3314 - d_loss: 0.4542 - v_ccc: 0.4459 - a_ccc: 0.6686 - d_ccc: 0.5458 - val_loss: 1.8620 - val_v_loss: 0.8260 - val_a_loss: 0.4631 - val_d_loss: 0.5729 - val_v_ccc: 0.1740 - val_a_ccc: 0.5369 - val_d_ccc: 0.4271
Epoch 8/50
6400/6400 [

Epoch 13/50
6400/6400 [==============================] - 4s 550us/step - loss: 1.1851 - v_loss: 0.4528 - a_loss: 0.3115 - d_loss: 0.4208 - v_ccc: 0.5472 - a_ccc: 0.6885 - d_ccc: 0.5792 - val_loss: 1.8843 - val_v_loss: 0.8359 - val_a_loss: 0.4548 - val_d_loss: 0.5936 - val_v_ccc: 0.1641 - val_a_ccc: 0.5452 - val_d_ccc: 0.4064
Epoch 14/50
6400/6400 [==============================] - 3s 526us/step - loss: 1.1734 - v_loss: 0.4441 - a_loss: 0.3125 - d_loss: 0.4168 - v_ccc: 0.5559 - a_ccc: 0.6875 - d_ccc: 0.5832 - val_loss: 1.8624 - val_v_loss: 0.8231 - val_a_loss: 0.4602 - val_d_loss: 0.5791 - val_v_ccc: 0.1769 - val_a_ccc: 0.5398 - val_d_ccc: 0.4209
Epoch 15/50
6400/6400 [==============================] - 3s 545us/step - loss: 1.1595 - v_loss: 0.4311 - a_loss: 0.3108 - d_loss: 0.4176 - v_ccc: 0.5689 - a_ccc: 0.6892 - d_ccc: 0.5824 - val_loss: 1.8452 - val_v_loss: 0.8068 - val_a_loss: 0.4599 - val_d_loss: 0.5786 - val_v_ccc: 0.1932 - val_a_ccc: 0.5401 - val_d_ccc: 0.4214
Epoch 16/50
6400/64

Epoch 17/50
6400/6400 [==============================] - 4s 551us/step - loss: 1.1664 - v_loss: 0.4384 - a_loss: 0.3112 - d_loss: 0.4168 - v_ccc: 0.5616 - a_ccc: 0.6888 - d_ccc: 0.5832 - val_loss: 1.8585 - val_v_loss: 0.8326 - val_a_loss: 0.4483 - val_d_loss: 0.5775 - val_v_ccc: 0.1674 - val_a_ccc: 0.5517 - val_d_ccc: 0.4225
Epoch 18/50
6400/6400 [==============================] - 3s 535us/step - loss: 1.1590 - v_loss: 0.4375 - a_loss: 0.3071 - d_loss: 0.4143 - v_ccc: 0.5625 - a_ccc: 0.6929 - d_ccc: 0.5857 - val_loss: 1.8402 - val_v_loss: 0.8203 - val_a_loss: 0.4607 - val_d_loss: 0.5592 - val_v_ccc: 0.1797 - val_a_ccc: 0.5393 - val_d_ccc: 0.4408
Epoch 19/50
6400/6400 [==============================] - 3s 536us/step - loss: 1.1517 - v_loss: 0.4296 - a_loss: 0.3099 - d_loss: 0.4122 - v_ccc: 0.5704 - a_ccc: 0.6901 - d_ccc: 0.5878 - val_loss: 1.8579 - val_v_loss: 0.8106 - val_a_loss: 0.4723 - val_d_loss: 0.5750 - val_v_ccc: 0.1894 - val_a_ccc: 0.5277 - val_d_ccc: 0.4250
Epoch 20/50
6400/64

Epoch 18/50
6400/6400 [==============================] - 3s 543us/step - loss: 1.1348 - v_loss: 0.4320 - a_loss: 0.2983 - d_loss: 0.4045 - v_ccc: 0.5680 - a_ccc: 0.7017 - d_ccc: 0.5955 - val_loss: 1.8735 - val_v_loss: 0.8100 - val_a_loss: 0.4794 - val_d_loss: 0.5840 - val_v_ccc: 0.1900 - val_a_ccc: 0.5206 - val_d_ccc: 0.4160
Epoch 19/50
2039/2039 [==============================] - 0s 175us/step
[0.18929226696491241, 0.591193675994873, 0.45227667689323425]
18
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 9s 1ms/step - loss: 1.8191 - v_loss: 0.8871 - a_loss: 0.4073 - d_loss: 0.5247 - v_ccc: 0.1129 - a_ccc: 0.5927 - d_ccc: 0.4753 - val_loss: 2.6284 - val_v_loss: 0.9281 - val_a_loss: 0.8590 - val_d_loss: 0.8413 - val_v_ccc: 0.0719 - val_a_ccc: 0.1410 - val_d_ccc: 0.1587
Epoch 2/50
6400/6400 [==============================] - 4s 548us/step - loss: 1.6018 - v_loss: 0.7499 - a_loss: 0.3781 - d_loss: 0.4738 - v_ccc: 0.2501 - a_ccc: 0.62